In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/2023/1-2th_month/Crush_SoMa/skip-gram/SM_idanBank

/content/drive/MyDrive/2023/1-2th_month/Crush_SoMa/skip-gram/SM_idanBank


In [ ]:
#데이터 전처리
from math import *
import numpy as np
import os 

train_text = open('dataset.train.txt','w')
test_text = open('dataset.test.txt','w')
valid_text = open('dataset.valid.txt','w')

text_files = []

folder_list = os.listdir('text')

for folder in folder_list:
    if(folder == '.DS_Store'):
        continue
    file_list = os.listdir('text/'+folder)
    text_files += [ ('text/'+folder+'/'+x) for x in file_list if x.endswith('.txt')]
    
data_size = []
data_size += [(0,ceil(len(text_files)*(7/10)))]
data_size += [(ceil(len(text_files)*(7/10))+1,ceil(len(text_files)*(9/10)))]
data_size += [(ceil(len(text_files)*(9/10))+1,len(text_files))]

for i, file in enumerate(text_files):
    readfile = open(file, 'r')
    while True:
        line = readfile.readline()
        if not line: break
        if i in range(data_size[0][0],data_size[0][1]):
            train_text.write(line)
        if i in range(data_size[1][0],data_size[1][1]):
            valid_text.write(line)
        if i in range(data_size[2][0],data_size[2][1]):
            test_text.write(line)
    readfile.close()
    
train_text.close()
valid_text.close()
test_text.close()

In [ ]:
!head -n 3 /proc/meminfo

MemTotal:       13297200 kB
MemFree:        11632128 kB
MemAvailable:   12327368 kB


In [ ]:
import numpy as np
import pickle
from trainer import Trainer
from optimizer import Adam
from skip_gram import SkipGram
from util import create_contexts_target, to_cpu
import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 1
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)



In [ ]:
id_to_word

In [ ]:

# 모델 등 생성
model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)



In [ ]:
# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()



In [ ]:
# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)